# TEMA 2: MODELO CLASIFICACIÓN CHURN
----

Vamos a recuperar el dataset procesado resultante del notebook del tema 1 (`T1_preprocesamiento.ipynb`): `churn_processed.csv`.

El dataset contiene información relativa a los clientes de una compañía telefónica con la columna "target" que indica si el cliente abandonó la compañía durante el mes siguiente. El objetivo es entrenar un modelo de clasificación que prediga la probabilidad de abandono de los clientes.

In [ ]:
#%pip install kds
#%pip install optuna
#%pip install optuna-integration

In [ ]:
# Importamos librerias

import pandas as pd
import numpy as np
from kds.metrics import plot_cumulative_gain
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import optuna.visualization as vis
from sklearn.metrics import auc, roc_curve
import pickle

In [ ]:
def predict_and_get_auc(model):
    
    y_train_prob = model.predict_proba(X_train)
    y_test_prob = model.predict_proba(X_test)

    fpr, tpr, threshold = roc_curve(y_train, y_train_prob[:, 1])
    print("AUC train = ", round(auc(fpr, tpr), 2))

    fpr, tpr, threshold = roc_curve(y_test, y_test_prob[:, 1])
    print("AUC test = ", round(auc(fpr, tpr), 2))

In [ ]:
df = pd.read_csv("churn_processed.csv")
df.head()

In [ ]:
y = df["target"]
X = df.drop(columns=["target"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

### Árbol de clasificación

In [ ]:
tree =  DecisionTreeClassifier(max_depth=4, random_state = 0)

tree.fit(X_train, y_train)

In [ ]:
predict_and_get_auc(tree)

El modelo tiene una capacidad predictiva muy alta. Además, obtenemos el mismo AUC en train y en test, por lo que no hay sobreajuste.

In [ ]:
plt.figure(figsize=(20, 15))
plot_tree(tree, feature_names=X_train.columns, filled = True)
plt.show()

### Optimización de hiperparámetros con optuna

Probamos la librería Optuna para la optimización de hiperparámetros de un random forest. Esta realiza una búsqueda inteligente: en lugar de probar todas las combinaciones posibles, va tomando decisiones secuenciales sobre qué combinaciones probar (con un máximo de intentos especificados) en base a los resultados de las iteraciones anteriores.

1. Función a optimizar: Se buscará maximizar el AUC

   - Espacio de búsqueda de hiperparámetros a optimizar.
   - Creación del modelo específico con los hiperparámetros sugeridos
   - Entrenamiento + cálculo de métrica

2. Creación del caso de estudio y especificación del nº de búsquedas a realizar.

3. Una vez acabada la búsqueda: obtenemos mejores hiperparámetros, entrenamos el modelo final y lo evaluamos sobre los datos de test.

In [ ]:
def objective(trial):

    # Definir el espacio de búsqueda de hiperparámetros
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 50)

    # Definir y entrenar el modelo con los hiperparámetros sugeridos
    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf)

    model.fit(X_train, y_train)

    # Calcular AUC obtenido en la iteración
    y_pred_test = model.predict_proba(X_test)
    fpr, tpr, threshold = roc_curve(y_test, y_pred_test[:, 1])
    auc_test = auc(fpr, tpr)
    
    return auc_test

# Crear un estudio Optuna y ejecutar la optimización
optuna.logging.set_verbosity(optuna.logging.ERROR)# desactivar logs para que quede más limpio
study = optuna.create_study(direction='maximize') # queremos maximizar la métrica (auc)
study.optimize(objective, n_trials=10) # se lanza la busqueda con 10 intentos

In [ ]:
study.best_params # mejores hiperparametros obtenidos

In [ ]:
optuna.visualization.plot_slice(study, target_name='AUC test') # visualización de los intentos y el AUC obtenido

In [ ]:
study.trials_dataframe() # obtener dataframe con el detalle de los intentos realizados

In [ ]:
tree_opt = DecisionTreeClassifier(**study.best_params)

tree_opt.fit(X_train, y_train)

predict_and_get_auc(tree_opt)

Nos quedamos con el Random Forest optimizado con optuna. Una vez entrenado el modelo, lo guardaríamos en formato pickle y lo utilizaríamos para hacer predicciones sobre nuevos datos de clientes que queremos estimar si son propensos a darse de baja:

In [ ]:
pickle.dump(tree_opt, open('modelo.pkl', 'wb'))

## Importancia de variables

In [ ]:
imp_df = pd.DataFrame({"variable": X.columns, "importancia relativa": tree_opt.feature_importances_}) \
.sort_values(by='importancia relativa', ascending = False)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(data=imp_df, x='importancia relativa', y='variable')
plt.title('Importancia de las Variables')
plt.xlabel('Importancia')
plt.ylabel('Características')
plt.show()

Si nos fijamos en la importancia relativa, vemos que aquellas variables con mayor importancia se corresponden con algunas de las que vimos gráficamente tenían alta relación con el target en el notebook `T1_preprocesamiento.ipynb`:

In [ ]:
sns.barplot(df, x = 'target', y = 'num_dt', errorbar=None)

In [ ]:
sns.barplot(df, x = 'target', y = 'incidencia', errorbar=None)

In [ ]:
sns.barplot(df, x = 'target', y = 'descuentos', errorbar=None)

### Curva de ganancia acumulada

El modelo devuelve probabilidades asignadas a cada cliente, lo que permite ordenar de mayor a menor probabilidad y tomar acciones solo sobre aquellos con probabilidad más alta. Muchas veces el volumen de clientes a accionar va a depender de presupuestos de negocio: por ejemplo, si solo hay presupuesto para llamar a 1000 clientes, se llamará a los 1000 con mayor probabilidad.

Sin embargo, cuando no está muy claro dónde hacer el corte, este gráfico puede ayudar a tomar decisiones:

In [ ]:
y_test_prob_tree  = tree_opt.predict_proba(X_test)

In [ ]:
plot_cumulative_gain(y_test, y_test_prob_tree[:,1])

La curva de ganancia acumulada indica el % de verdaderos positivos detectados por el modelo si seleccionásemos al x% de mayor probabilidad.

- Eje x: % acumulado de observaciones (de mayor a menor probabilidad)
- Eje y: % acumulado de verdaderos positivos


La curva "Wizard" sería la curva del modelo perfecto (aquel que separase perfectamente): hay que fijarse en la curva "Model"

**En este caso la curva indica que tan solo contactando a los 3 primeros deciles (el 30% de clientes con mayor probabilidad dada por el modelo), conseguiríamos detectar más del 80% de los que se acaban dando de baja. Sin usar modelo, contactando a un 30% aleatorio detectarías solo un 30% de las bajas. Por tanto, el modelo te permite afinar contactando solo a aquellos clientes que estás más seguro que se van a dar de baja, y por tanto ahorrar costes**

## Carga de nuevos datos y predicciones

Supongamos que tenemos un nuevo set de datos con los clientes en cartera actualizados a fecha de hoy y queremos obtener la probabilidad de que se den de baja. Este dataset ya ha sido preparado en un script previo, por lo que directamente leemos este dataset y aplicamos predicciones cargando el modelo entrenado:

In [ ]:
df_new = pd.read_csv('churn_new_data.csv')
df_new.head() # queremos predecir la probabilidad de baja de estos nuevos clientes

In [ ]:
modelo = pickle.load(open('modelo.pkl', 'rb')) # cargamos el modelo entrenado previamente

In [ ]:
preds = modelo.predict_proba(df_new.drop('id', axis = 1))[:,1]

In [ ]:
preds

In [ ]:
df_new['pred'] = preds # asignamos cada probabilidad al idcliente correspondiente para poder accionarlo

In [ ]:
df_new.sort_values(by='pred', ascending = False).head()